In [9]:
from typing import List

import numpy as np
from environs import Env
import cv2

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.python.keras.engine import training
from tensorflow.keras.layers import (
    ZeroPadding2D,
    Input,
    Conv2D,
    BatchNormalization,
    PReLU,
    Add,
    Dropout,
    Flatten,
    Dense,
)

env = Env()
env.read_env('../.env', recurse=False)

True

Напишем программу для использования ArcFace без библиотеки

In [3]:
def load_model(
    url="https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5",
) -> Model:
    """
    Construct ArcFace model, download its weights and load
    Returns:
        model (Model)
    """
    base_model = ResNet34()
    inputs = base_model.inputs[0]
    arcface_model = base_model.outputs[0]
    arcface_model = BatchNormalization(momentum=0.9, epsilon=2e-5)(arcface_model)
    arcface_model = Dropout(0.4)(arcface_model)
    arcface_model = Flatten()(arcface_model)
    arcface_model = Dense(512, activation=None, use_bias=True, kernel_initializer="glorot_normal")(
        arcface_model
    )
    embedding = BatchNormalization(momentum=0.9, epsilon=2e-5, name="embedding", scale=True)(
        arcface_model
    )
    model = Model(inputs, embedding, name=base_model.name)

    # ---------------------------------------
    # check the availability of pre-trained weights

    # home = folder_utils.get_deepface_home()

    file_name = "../models/arcface_weights.h5"

    # if os.path.isfile(file_name) != True:

    #     logger.info(f"{file_name} will be downloaded to {file_name}")
    #     gdown.download(url, output, quiet=False)

    # ---------------------------------------

    model.load_weights(file_name)

    return model

In [4]:
def ResNet34() -> Model:
    """
    ResNet34 model
    Returns:
        model (Model)
    """
    img_input = Input(shape=(112, 112, 3))

    x = ZeroPadding2D(padding=1, name="conv1_pad")(img_input)
    x = Conv2D(
        64, 3, strides=1, use_bias=False, kernel_initializer="glorot_normal", name="conv1_conv"
    )(x)
    x = BatchNormalization(axis=3, epsilon=2e-5, momentum=0.9, name="conv1_bn")(x)
    x = PReLU(shared_axes=[1, 2], name="conv1_prelu")(x)
    x = stack_fn(x)

    model = training.Model(img_input, x, name="ResNet34")

    return model

In [5]:
def block1(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
    bn_axis = 3

    if conv_shortcut:
        shortcut = Conv2D(
            filters,
            1,
            strides=stride,
            use_bias=False,
            kernel_initializer="glorot_normal",
            name=name + "_0_conv",
        )(x)
        shortcut = BatchNormalization(
            axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + "_0_bn"
        )(shortcut)
    else:
        shortcut = x

    x = BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + "_1_bn")(x)
    x = ZeroPadding2D(padding=1, name=name + "_1_pad")(x)
    x = Conv2D(
        filters,
        3,
        strides=1,
        kernel_initializer="glorot_normal",
        use_bias=False,
        name=name + "_1_conv",
    )(x)
    x = BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + "_2_bn")(x)
    x = PReLU(shared_axes=[1, 2], name=name + "_1_prelu")(x)

    x = ZeroPadding2D(padding=1, name=name + "_2_pad")(x)
    x = Conv2D(
        filters,
        kernel_size,
        strides=stride,
        kernel_initializer="glorot_normal",
        use_bias=False,
        name=name + "_2_conv",
    )(x)
    x = BatchNormalization(axis=bn_axis, epsilon=2e-5, momentum=0.9, name=name + "_3_bn")(x)

    x = Add(name=name + "_add")([shortcut, x])
    return x


def stack1(x, filters, blocks, stride1=2, name=None):
    x = block1(x, filters, stride=stride1, name=name + "_block1")
    for i in range(2, blocks + 1):
        x = block1(x, filters, conv_shortcut=False, name=name + "_block" + str(i))
    return x


def stack_fn(x):
    x = stack1(x, 64, 3, name="conv2")
    x = stack1(x, 128, 4, name="conv3")
    x = stack1(x, 256, 6, name="conv4")
    return stack1(x, 512, 3, name="conv5")

In [6]:
from abc import ABC, abstractmethod
from typing import Any, Union, List, Tuple

class FacialRecognition(ABC):
    model: Union[Model, Any]
    model_name: str
    input_shape: Tuple[int, int]
    output_shape: int


    @abstractmethod
    def find_embeddings(self, img: np.ndarray) -> List[float]:
        pass

In [7]:
class ArcFaceClient(FacialRecognition):
    """
    ArcFace model class
    """

    def __init__(self):
        self.model = load_model()
        self.model_name = "ArcFace"
        self.input_shape = (112, 112)
        self.output_shape = 512

    def find_embeddings(self, img: np.ndarray) -> List[float]:
        """
        find embeddings with ArcFace model
        Args:
            img (np.ndarray): pre-loaded image in BGR
        Returns
            embeddings (list): multi-dimensional vector
        """
        # model.predict causes memory issue when it is called in a for loop
        # embedding = model.predict(img, verbose=0)[0].tolist()
        return self.model(img, training=False).numpy()[0].tolist()

In [8]:
model = ArcFaceClient()

2024-02-28 13:08:19.299991: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-28 13:08:19.325231: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [23]:
import matplotlib.pyplot as plt

img = cv2.imread('../datasets/facescrub_images/Adam_Brody/Adam_Brody_241.png')
img = np.expand_dims(img, axis=0)

model.find_embeddings(img)

[-191.85186767578125,
 628.3995971679688,
 16.363235473632812,
 -565.1090087890625,
 -125.89778137207031,
 118.6379165649414,
 197.51853942871094,
 116.16345977783203,
 -173.35348510742188,
 122.5489273071289,
 -13.882956504821777,
 511.5950012207031,
 384.61328125,
 -99.73631286621094,
 -696.66650390625,
 16.09794044494629,
 -437.02154541015625,
 -390.82562255859375,
 120.58177947998047,
 -304.8488464355469,
 -771.7843017578125,
 179.36422729492188,
 -226.230224609375,
 -232.35838317871094,
 -68.7524642944336,
 190.2395782470703,
 9.170049667358398,
 -198.6533660888672,
 -331.3713073730469,
 396.45941162109375,
 193.2986602783203,
 141.04116821289062,
 -171.50791931152344,
 135.73011779785156,
 373.2608642578125,
 -120.51568603515625,
 193.10728454589844,
 293.6642150878906,
 -242.27362060546875,
 187.28968811035156,
 6.7915730476379395,
 92.72702026367188,
 -839.6629638671875,
 -81.5510025024414,
 -640.9343872070312,
 143.1265411376953,
 54.01491928100586,
 220.4518280029297,
 -99.53

In [21]:
from deepface import DeepFace

len(DeepFace.represent(img1, model_name="ArcFace")[0]['embedding'])

512